In [8]:
import model_lib
import numpy as np
import warnings
warnings.filterwarnings('ignore', '.*output shape of zoom.*')
import pickle
import importlib
importlib.reload(model_lib)
import os
import time
from PIL import Image

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


In [9]:
# config to train
# TODO: check Config is correct
class ProposalConfig():
    NAME = "InSegm"
    GPU_COUNT = 1
    # online training
    IMAGES_PER_GPU = 1
    STEPS_PER_EPOCH = 100
    # not going to use these
    N_DISTORTIONS = 0
    MAX_DISTORTION = 0.3
    MIN_DISTORTION = -0.1
    
    VALIDATION_STEPS = 20
    # including gt
    NUM_CLASSES = 81
    # only flips
    IMAGE_AUGMENT = True

    MEAN_PIXEL = np.array([123.7, 116.8, 103.9],dtype=np.float32)
    MAX_GT_INSTANCES = 100
    DETECTION_MAX_INSTANCES = 100
    DETECTION_MIN_CONFIDENCE = 0.7
    CLASS_NAMES = [
        'BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
        'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
        'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
        'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella',
        'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
        'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
        'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork',
        'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
        'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
        'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
        'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
        'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
        'scissors', 'teddy bear', 'hair drier', 'toothbrush'
    ]
    DETECTION_NMS_THRESHOLD = 0.3
    LEARNING_RATE = 0.05
    LEARNING_MOMENTUM = 0.9
    WEIGHT_DECAY = 0.0001
    WIDTH = 224
    HEIGHT = 224
    MASK_SHAPE = (64,64)
    GRID_WIDTH = 16
    GRID_HEIGHT = 16
    CLUE_SHAPE = (20,20)
    GRID_SHAPE = (GRID_WIDTH, GRID_HEIGHT)
    GRID_RESOLUTION = (1, 1)
    IS_PADDED = True
    MASK_THRESOLD = 0.7
    def __init__(self):
        self.BATCH_SIZE = self.IMAGES_PER_GPU * self.GPU_COUNT
        self.IMAGE_SHAPE = (self.WIDTH, self.HEIGHT,3)
        self.MAX_BATCH_SIZE = self.BATCH_SIZE*32

    def display(self):
        """Display Configuration values."""
        print("\nConfigurations:")
        for a in dir(self):
            if not a.startswith("__") and not callable(getattr(self, a)):
                print("{:30} {}".format(a, getattr(self, a)))
        print("\n")

In [10]:
root_dir = "/media/data/nishanth/aravind/"
config = ProposalConfig()
model_dir = "./models/"
train_dataset,val_dataset = None,None
train_pickle = root_dir+"val_cid.pickle"
val_pickle = root_dir+"val_cid.pickle"
class ClassInstancesDataset():
    def __init__(self):
        self.class_wise_instance_info = [[] for i in range(81)]
        self.instance_info = []
import time
start = time.time()
with open(train_pickle,"rb") as train_ann:
    train_cid = pickle.load(train_ann)
end = time.time()
print(end-start)
#     train_dataset = model_lib.CocoDataset(train_cid,config)
with open(val_pickle,"rb") as val_ann:
    val_cid = pickle.load(val_ann)
#     val_dataset = model_lib.CocoDataset(val_cid,config)
train_loader = model_lib.get_loader(train_cid,config,"ins")
val_loader = model_lib.get_loader(val_cid,config,"ins")

0.4214510917663574


In [ ]:
import torch
import torch.nn.functional as F
net = model_lib.SimpleHGModel()
net.load_state_dict(torch.load(model_dir+"model_bce_0_7600.pt"))
net = net.cuda()
with torch.no_grad():
    for i,data in enumerate(val_loader):
        batch_images,batch_impulses,batch_gt_responses,batch_class_ids = data
        a,b,c= batch_images.numpy(), batch_impulses.numpy(),batch_gt_responses.numpy()
        a = np.moveaxis(a,1,-1)
        b = np.moveaxis(b,1,-1)
        c = np.moveaxis(c,1,-1)
        Image.fromarray((a[0]*128 + config.MEAN_PIXEL).astype(np.uint8),"RGB").show()
        Image.fromarray((b[0][:,:,0]*128).astype(np.uint8),"L").show()
        Image.fromarray((c[0][:,:,0]*128).astype(np.uint8),"L").show()
        batch_images,batch_impulses,batch_gt_responses,batch_class_ids = batch_images.cuda(),batch_impulses.cuda(),batch_gt_responses.cuda(),batch_class_ids.cuda()
        pred_class = net([batch_images,batch_impulses])
#         print(pred_class)
        pred_class = F.softmax(pred_class,dim=-1).squeeze()
        maxs, indices = torch.topk(pred_class,5,-1)
        print(maxs.shape,indices.shape)
        for i in range(5):
            print(maxs[i],indices[i],config.CLASS_NAMES[int(indices[i])])
#         print(batch_class_ids)
#         print(indices)
#         print("gt_class:",config.CLASS_NAMES[int(batch_class_ids[0])])
#         print("pred_class: ",pred_class[int(batch_class_ids[0])])
#         print(config.CLASS_NAMES[int(indices[0])])
        input()

torch.Size([5]) torch.Size([5])
tensor(0.4362, device='cuda:0') tensor(1, device='cuda:0') person
tensor(0.1634, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(0.1622, device='cuda:0') tensor(0, device='cuda:0') BG
tensor(1.00000e-02 *
       3.5993, device='cuda:0') tensor(17, device='cuda:0') dog
tensor(1.00000e-02 *
       3.0619, device='cuda:0') tensor(61, device='cuda:0') dining table

torch.Size([5]) torch.Size([5])
tensor(0.7261, device='cuda:0') tensor(1, device='cuda:0') person
tensor(0.1053, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(1.00000e-02 *
       2.1138, device='cuda:0') tensor(8, device='cuda:0') truck
tensor(1.00000e-02 *
       1.8198, device='cuda:0') tensor(61, device='cuda:0') dining table
tensor(1.00000e-02 *
       1.7935, device='cuda:0') tensor(3, device='cuda:0') car

torch.Size([5]) torch.Size([5])
tensor(0.4731, device='cuda:0') tensor(1, device='cuda:0') person
tensor(0.1952, device='cuda:0') tensor(0, device='cuda:0') BG
tenso


torch.Size([5]) torch.Size([5])
tensor(0.3787, device='cuda:0') tensor(1, device='cuda:0') person
tensor(0.2775, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(0.1181, device='cuda:0') tensor(0, device='cuda:0') BG
tensor(1.00000e-02 *
       3.3373, device='cuda:0') tensor(8, device='cuda:0') truck
tensor(1.00000e-02 *
       3.2553, device='cuda:0') tensor(3, device='cuda:0') car

torch.Size([5]) torch.Size([5])
tensor(0.5319, device='cuda:0') tensor(1, device='cuda:0') person
tensor(0.1941, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(1.00000e-02 *
       6.3203, device='cuda:0') tensor(0, device='cuda:0') BG
tensor(1.00000e-02 *
       5.9853, device='cuda:0') tensor(3, device='cuda:0') car
tensor(1.00000e-02 *
       1.8357, device='cuda:0') tensor(61, device='cuda:0') dining table

torch.Size([5]) torch.Size([5])
tensor(0.5065, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(0.2955, device='cuda:0') tensor(1, device='cuda:0') person
tensor(1.000


torch.Size([5]) torch.Size([5])
tensor(0.5987, device='cuda:0') tensor(1, device='cuda:0') person
tensor(0.2000, device='cuda:0') tensor(0, device='cuda:0') BG
tensor(1.00000e-02 *
       6.9277, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(1.00000e-02 *
       2.2159, device='cuda:0') tensor(3, device='cuda:0') car
tensor(1.00000e-02 *
       1.8326, device='cuda:0') tensor(17, device='cuda:0') dog

torch.Size([5]) torch.Size([5])
tensor(0.6278, device='cuda:0') tensor(1, device='cuda:0') person
tensor(0.1166, device='cuda:0') tensor(0, device='cuda:0') BG
tensor(1.00000e-02 *
       9.6499, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(1.00000e-02 *
       3.0840, device='cuda:0') tensor(3, device='cuda:0') car
tensor(1.00000e-02 *
       2.2640, device='cuda:0') tensor(61, device='cuda:0') dining table

torch.Size([5]) torch.Size([5])
tensor(0.4242, device='cuda:0') tensor(1, device='cuda:0') person
tensor(0.2202, device='cuda:0') tensor(57, device='cuda:0'


torch.Size([5]) torch.Size([5])
tensor(0.4629, device='cuda:0') tensor(1, device='cuda:0') person
tensor(0.2159, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(0.1188, device='cuda:0') tensor(0, device='cuda:0') BG
tensor(1.00000e-02 *
       4.1415, device='cuda:0') tensor(3, device='cuda:0') car
tensor(1.00000e-02 *
       2.1642, device='cuda:0') tensor(8, device='cuda:0') truck

torch.Size([5]) torch.Size([5])
tensor(0.6345, device='cuda:0') tensor(1, device='cuda:0') person
tensor(0.1443, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(1.00000e-02 *
       5.4634, device='cuda:0') tensor(17, device='cuda:0') dog
tensor(1.00000e-02 *
       2.4253, device='cuda:0') tensor(3, device='cuda:0') car
tensor(1.00000e-02 *
       2.1117, device='cuda:0') tensor(8, device='cuda:0') truck

torch.Size([5]) torch.Size([5])
tensor(0.4513, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(0.2284, device='cuda:0') tensor(1, device='cuda:0') person
tensor(1.00000e-02

In [4]:
import random
import time
np.set_printoptions(threshold=np.nan)
counts = [0 for i in range(81)]
train_loader = iter(train_loader)
for i in range(1000):
    start = time.time()
    g = next(train_loader)
    print(time.time()-start)
for i,g in enumerate(train_loader):
    print("sample:" + str(i))
#     start = time.time()
    batch_images,batch_impulses,batch_gt_responses,batch_class_ids = g
#     print(batch_class_ids)
    for i in batch_class_ids:
        counts[int(i.item())] += 1
print(counts)
#     batch_images = np.moveaxis(batch_images.numpy(),1,-1)
#     print(batch_images.shape)
#     batch_impulses = batch_impulses.squeeze().numpy()*128
#     batch_gt_responses = batch_gt_responses.squeeze().numpy()*128
#     batch_class_ids = batch_class_ids.numpy()
#     # s =(time.time()-start)
#     # print(s)
#     batch_images[0] *= 128
#     batch_images[0] += config.MEAN_PIXEL
#     img = Image.fromarray((batch_images[0]).astype("uint8"),"RGB")
#     img.show()
#     print((np.sum(batch_gt_responses)//128)**0.5)
#     mask = Image.fromarray(((batch_gt_responses)).astype("uint8"),"L")
#     mask.show()
#     print(config.CLASS_NAMES[batch_class_ids[0]])
#     impulse = Image.fromarray(((batch_impulses)).astype("uint8"),"L")
#     impulse.show()
#     input()

5.8727030754089355
1.0894262790679932
0.7940988540649414
0.006885528564453125
0.007991790771484375
0.008913755416870117
0.02939128875732422
0.010446310043334961
0.0075948238372802734
0.006993293762207031
0.007034778594970703
0.009102106094360352
0.007321357727050781
0.0072498321533203125
0.0072939395904541016
0.007134437561035156
3.5132102966308594
1.0766754150390625
0.6235623359680176
0.010315656661987305
0.007702350616455078
0.012910842895507812
0.008159875869750977
0.00718235969543457
0.02111506462097168
0.007714271545410156
0.008462667465209961
0.008092164993286133
0.011322259902954102
0.010010242462158203
1.5816411972045898
0.007668018341064453
2.0190937519073486
1.1086020469665527
1.0073204040527344
0.025751352310180664
0.00955820083618164
0.00836324691772461
0.0075795650482177734
0.007452249526977539
0.010152816772460938
0.012168169021606445
0.009928703308105469
0.00722956657409668
0.012662649154663086
0.04663562774658203
0.6680276393890381
0.017316341400146484
1.54194974899292


Process Process-8:
Process Process-2:
Process Process-4:
Process Process-5:
Process Process-7:
Process Process-14:
Process Process-16:
Process Process-10:
Traceback (most recent call last):
Process Process-1:
Process Process-13:
Process Process-12:
Process Process-3:
Traceback (most recent call last):
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()

  File "/home/aravind/re/model_lib.py", line 270, in load_image_gt
    padding=config.IS_PADDED)
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/site-packages/skimage/io/_io.py", line 61, in imread
    img = call_plugin('imread', fname, plugin=plugin, **plugin_args)
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/site-packages/numpy/core/fromnumeric.py", line 1882, in sum
    out=out, **kwargs)
  File "/home/aravind/re/model_lib.py", line 263, in load_image_gt
    image = self.read_image(image_path)
  File "/home/aravind/re/model_lib.py", line 263, in load_image_gt
    image = self.read_image(image_path)
  File "/home/aravind/re/model_lib.py", line 229, in resize_image
    image, (round(h * scale), round(w * scale)))
Traceback (most recent call last):
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/site-packages/t

  File "/home/aravind/re/model_lib.py", line 175, in __getitem__
    image, masks, class_id = self.load_image_gt(instance_index)
  File "/home/aravind/re/model_lib.py", line 239, in resize_image
    image = np.pad(image, padding, mode='constant', constant_values=0)
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 57, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/site-packages/skimage/io/manage_plugins.py", line 211, in call_plugin
    return func(*args, **kwargs)
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/site-packages/scipy/misc/pilutil.py", line 564, in imresize
    imnew = im.resize(size, resample=func[interp])
  File "/home/aravind/re/model_lib.py", line 263, in load_image_gt
    image = self.read_image(image_path)
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/site-packages/PIL/Image.py", line 1749, in resize
   

Traceback (most recent call last):
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-72c191e434fa>", line 8, in <module>
    g = next(train_loader)
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 280, in __next__
    idx, batch = self._get_batch()
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 259, in _get_batch
    return self.data_queue.get()
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/multiprocessing/connection

  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/site-packages/numpy/lib/utils.py", line 101, in newfunc
    return func(*args, **kwds)
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/site-packages/numpy/lib/arraypad.py", line 1084, in _validate_lengths
    if (chk[0] < 0) or (chk[1] < 0):
  File "/home/aravind/re/model_lib.py", line 175, in __getitem__
    image, masks, class_id = self.load_image_gt(instance_index)
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/site-packages/pycocotools/mask.py", line 89, in decode
    return _mask.decode(rleObjs)
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/site-packages/scipy/misc/pilutil.py", line 564, in imresize
    imnew = im.resize(size, resample=func[interp])
  File "/home/aravind/re/model_lib.py", line 270, in load_image_gt
    padding=config.IS_PADDED)
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/site-packages/PIL/Image.py", line 1220, in getdata
    self.load()
  File "/home/aravind/re/mode

KeyboardInterrupt: 

KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/home/aravind/re/model_lib.py", line 229, in resize_image
    image, (round(h * scale), round(w * scale)))
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/site-packages/PIL/ImageFile.py", line 231, in load
    n, err_code = decoder.decode(b)
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/site-packages/numpy/lib/utils.py", line 101, in newfunc
    return func(*args, **kwds)
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/site-packages/skimage/io/_plugins/pil_plugin.py", line 37, in imread
    return pil_to_ndarray(im, dtype=dtype, img_num=img_num)
KeyboardInterrupt
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/site-packages/skimage/io/_plugins/pil_plugin.py", line 53, in pil_to_ndarray
    im.getdata()[0]
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/site-packages/scipy/misc/pilutil.py", line 564, in imresize
    imnew = im.resize(size, resample=func[interp])
  File "/home/ara

In [ ]:
[len(c) for c in train_cid.class_wise_instance_info]